In [2]:
import numpy as np
from scipy.spatial.transform import Rotation as R
import open3d as o3d

In [3]:
fx = 912.3776245117188
fy = 911.2467651367188
cx = 637.6239013671875
cy = 359.3946533203125

In [4]:
# get transformation matrix for depth map to xyz coordinates

transformation_matrix = np.array(
    [
        [fx, 0, cx],
        [0, fy, cy],
        [0, 0, 1],
    ]
)

In [13]:
# load depth map
sample = "Duran_250mL_80ml_1105_2559"
sample = "Cell_Flask_400mL_414ml_0905_3539"
sample = "Pyrex_100mL_45ml_1105_5713"
sample = "Pyrex_100mL_95ml_0905_2157"
depth_map_vessel = np.load(
    "../../volume_estimation/data/processed/"
    + sample
    + "/Input_EmptyVessel_Depth_segmented.npy"
)
depth_map_liquid = np.load(
    "../../volume_estimation/data/processed/"
    + sample
    + "/Input_ContentDepth_segmented.npy"
)
img = np.load(
    "../../volume_estimation/data/processed/" + sample + "/Input_RGBImage.npy"
)

# convert depth map from log to linear space
depth_map_vessel = np.exp(depth_map_vessel)
depth_map_liquid = np.exp(depth_map_liquid)

# volume_estimation\data\processed\Cell_Flask_400mL_414ml_0905_3539
# volume_estimation\data\processed\Pyrex_100mL_45ml_1105_5713
# volume_estimation\data\processed\Pyrex_100mL_95ml_0905_2157

In [14]:
def depth_map_conversion(depth_map, camera_intrinsics):
    # Get the intrinsic parameters
    fx = camera_intrinsics[0][0]
    fy = camera_intrinsics[1][1]
    cx = camera_intrinsics[0][2]
    cy = camera_intrinsics[1][2]
    print(fx, fy, cx, cy)

    # Create the meshgrid
    rows, cols = depth_map.shape[:2]
    print(rows, cols)
    c, r = np.meshgrid(np.arange(cols), np.arange(rows), sparse=True)

    # Calculate the 3D coordinates
    x = (c - cx) * depth_map / fx
    print(x.shape)
    # value count in x
    unique, counts = np.unique(x, return_counts=True)
    print(dict(zip(unique, counts)))
    y = (r - cy) * depth_map / fy
    z = depth_map

    # Stack the coordinates and return them
    return x, y, z

In [15]:
def Depth2XYZ(Img, DepthMap, transformation_matrix):
    # M = K.intrinsic_matrix
    M = transformation_matrix
    height = DepthMap.shape[0]
    width = DepthMap.shape[1]
    GridY = np.array(list(range(height))) - height / 2
    GridY = np.transpose(np.tile(GridY, (width, 1)))
    GridX = np.array(list(range(width))) - width / 2
    GridX = np.tile(GridX, (height, 1))
    XYZ = np.zeros([height, width, 3], dtype=np.float32)
    XYZ[:, :, 2] = DepthMap
    XYZ[:, :, 1] = DepthMap * GridY / M[0, 0]
    XYZ[:, :, 0] = DepthMap * GridX / M[1, 1]
    ROI = (DepthMap > 1).astype(np.float32)
    return XYZ, ROI, Img, DepthMap

In [21]:
xyzMap, ROI, RGB, depth = Depth2XYZ(img, depth_map_vessel, transformation_matrix)
xyzMap_liquid, ROI_liquid, RGB_liquid, depth_liquid = Depth2XYZ(
    img, depth_map_liquid, transformation_matrix
)

In [17]:
def DisplayPointCloud(xyzMap, RGB, ROI, step=1):
    NumPoints = int(xyzMap.shape[0] * xyzMap.shape[1] / step**2)
    xyz = np.zeros([NumPoints, 3], np.float32)
    colors = np.zeros([NumPoints, 3], np.float32)
    tt = 0
    for x in range(0, xyzMap.shape[1], step):
        for y in range(0, xyzMap.shape[0], step):
            if (ROI[y, x]) > 0 and tt < NumPoints:
                xyz[tt] = xyzMap[y, x]
                colors[tt] = RGB[y, x] / 255
                tt += 1
            # print(tt)
    xyz = xyz[:tt, :]
    colors = colors[:tt, :]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    o3d.visualization.draw_geometries([pcd])

In [19]:
DisplayPointCloud(xyzMap, RGB, ROI, step=1)
DisplayPointCloud(xyzMap_liquid, RGB_liquid, ROI_liquid, step=1)

In [22]:
import open3d as o3d
import numpy as np


def DisplayPointCloud(xyzMap, RGB, ROI, step=1):
    NumPoints = int(xyzMap.shape[0] * xyzMap.shape[1] / step**2)
    xyz = np.zeros([NumPoints, 3], np.float32)
    colors = np.zeros([NumPoints, 3], np.float32)
    tt = 0
    for x in range(0, xyzMap.shape[1], step):
        for y in range(0, xyzMap.shape[0], step):
            if ROI[y, x] > 0 and tt < NumPoints:
                xyz[tt] = xyzMap[y, x]
                colors[tt] = [0.5, 0.5, 0.5]  # Assign gray color
                tt += 1
    xyz = xyz[:tt, :]
    colors = colors[:tt, :]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    o3d.visualization.draw_geometries([pcd])


DisplayPointCloud(xyzMap, RGB, ROI)

In [23]:
import open3d as o3d
import numpy as np


def DisplayPointCloud(xyzMap, RGB, ROI, step=1):
    NumPoints = int(xyzMap.shape[0] * xyzMap.shape[1] / step**2)
    xyz = np.zeros([NumPoints, 3], np.float32)
    colors = np.zeros([NumPoints, 3], np.float32)
    tt = 0
    for x in range(0, xyzMap.shape[1], step):
        for y in range(0, xyzMap.shape[0], step):
            if ROI[y, x] > 0 and tt < NumPoints:
                xyz[tt] = xyzMap[y, x]
                colors[tt] = RGB[y, x] / 255
                tt += 1
    xyz = xyz[:tt, :]
    colors = colors[:tt, :]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    # Create a colormap similar to glass
    colormap = o3d.visualization.create_colormap("jet")

    # Assign the colormap to the point cloud
    pcd.colors = o3d.utility.Vector3dVector(colormap[colors[:, 0], :])

    o3d.visualization.draw_geometries([pcd])


DisplayPointCloud(xyzMap, RGB, ROI)

AttributeError: module 'open3d.visualization' has no attribute 'create_colormap'